In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy import stats

# ---------------- USER INPUTS ----------------
img_path = "C1f.jpg"     # image of intensity vs. radius
physical_length_cm = 3.28         # physical span in cm - C1 3.28cm C2 4.592
lambda_m = 632.8e-9              # red laser wavelength (HeNe or diode)
sigma_smooth = 2.5               # Gaussian smoothing strength (1–5 typical) 
min_peak_prominence = 0.05       # reject small bumps
min_peak_distance_frac = 0.05   # fraction of total length between peaks
# --------------------------------------------

# Load grayscale image
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError("Image not found.")

# Collapse vertically to 1D intensity
profile = img.mean(axis=0)
profile = (profile - profile.min()) / (profile.max() - profile.min())

# Smooth intensity
smoothed = gaussian_filter1d(profile, sigma=sigma_smooth)

# Convert pixel positions to distances
pixels = np.arange(len(smoothed))
length_m = physical_length_cm / 100.0
pixel_size_m = length_m / len(smoothed)
r_m = pixels * pixel_size_m

# Estimate approximate fringe spacing for adaptive distance
approx_period = len(smoothed) * min_peak_distance_frac
peaks, props = find_peaks(
    smoothed,
    distance=approx_period,
    prominence=min_peak_prominence
)
r_peaks = r_m[peaks]

# Optionally remove first peak if too close to center
if len(r_peaks) > 2 and (r_peaks[1] - r_peaks[0]) < 0.5 * np.median(np.diff(r_peaks[1:])):
    r_peaks = r_peaks[1:]
    peaks = peaks[1:]

# Assign fringe orders
m = np.arange(len(r_peaks))

# Fit r^2 = (2 λ Q) m + c
y = r_peaks**2
slope, intercept, r_value, p_value, stderr = stats.linregress(m, y)
Q = slope / (2 * lambda_m)
Q_err = stderr / (2 * lambda_m)

# Report results
print(f"Detected {len(peaks)} fringes")
print(f"Slope = {slope:.3e} m^2/order")
print(f"Q = R1*R2/(R2-R1) = {Q:.3e} ± {Q_err:.3e} m")
print(f"Fit R² = {r_value**2:.4f}")

# Plot
plt.style.use('seaborn-v0_8-whitegrid')  
plt.figure(figsize=(8,6))
plt.plot(r_m*1e3, profile, color='gray', alpha=0.4, label='Raw profile')
plt.plot(r_m*1e3, smoothed, 'b-', lw=1.5, label='Smoothed')
plt.plot(r_peaks*1e3, smoothed[peaks], 'ro', label='Detected peaks')
plt.xlabel("Radius (mm)")
plt.ylabel("Normalized intensity")
plt.title("Intensity profile — sample 1")
plt.legend()
plt.tight_layout()
plt.savefig('s1.png', dpi=300)
plt.show()